In [1]:
file_name = "combined_sampling"
save_directory = "analysis_result"
report = {}

In [2]:
import os, sys
cwd = os.getcwd()
print("Current working directory:", cwd)
home_dir = os.path.dirname(os.path.dirname(cwd))
print("Home directory:", home_dir)
sys.path.append(home_dir)

Current working directory: /home/hoon/dd-agent/llm_dd/example/test_case
Home directory: /home/hoon/dd-agent/llm_dd


In [3]:
import json
import pandas as pd
from rdkit.Chem import QED
from rdkit import Chem
from agentD.analysis.analysis_helper import rule_based_evaluation
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Preliminary - File Concatenation

In [ ]:
names = ["Mol2Mol_sampling_update", "Reinvent_sampling_update"]

# Prepare containers to stack entries
all_original_properties = []
all_updated_properties = []
all_update_mappings = []

for n in names:
    print(f"Processing {n}...")

    # Define paths
    original_property_path = f"property/{n}.csv"
    updated_property_path = f"property/{n}_update.csv"
    update_mapping_path = f"refinement/{n}.csv"

    # Read and add 'Method' column to identify source
    df_original = pd.read_csv(original_property_path)
    df_original['Method'] = n
    all_original_properties.append(df_original)

    df_updated = pd.read_csv(updated_property_path)
    df_updated['Method'] = n + "_update"
    all_updated_properties.append(df_updated)

    df_mapping = pd.read_csv(update_mapping_path)
    df_mapping['Method'] = n
    all_update_mappings.append(df_mapping)

# Stack (concatenate) by type
stacked_original_df = pd.concat(all_original_properties, ignore_index=True)
stacked_updated_df = pd.concat(all_updated_properties, ignore_index=True)
stacked_mapping_df = pd.concat(all_update_mappings, ignore_index=True)

# Save all combined data
# os.makedirs("combined", exist_ok=True)
stacked_original_df.to_csv(f"property/{file_name}.csv", index=False)
stacked_updated_df.to_csv(f"property/{file_name}_update.csv", index=False)
stacked_mapping_df.to_csv(f"refinement/{file_name}.csv", index=False)

print("✅ Stacked files saved with 'Method' column in the save directory.")

Processing Mol2Mol_sampling_update...
Processing Reinvent_sampling_update...
✅ Stacked files saved with 'Method' column in the save directory.


# Update vs. Original

In [4]:
original_update_mapping_path = f"refinement/{file_name}.csv"
df = pd.read_csv(original_update_mapping_path)
invalid_count = 0
for index, row in df.iterrows():
    smiles = row['Updated_SMILES']
    if pd.isna(smiles):
        continue
    smiles = smiles.strip()
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        invalid_count += 1


print("-" * 50)
print(f"data number: {len(df)}")
print(f"invalid SMILES: {invalid_count}")
print("-" * 50)

report['method'] = file_name
report['data_number'] = len(df)
report['invalid_smiles'] = invalid_count

--------------------------------------------------
data number: 99
invalid SMILES: 4
--------------------------------------------------


[14:37:41] Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 21
[14:37:41] SMILES Parse Error: unclosed ring for input: 'CN(C)S(=O)(=O)NC(=O)c1ccc(N2CCN(C[C@H]3CC[C@@H](C(=O)O)CC3)[C@@H]2Oc2cnc(N)cc2Cl)'
[14:37:41] SMILES Parse Error: unclosed ring for input: 'COc1ccc(OCCNC(=O)c2cc3cc(O)ccc3n2Cc2ccccc2)'
[14:37:41] SMILES Parse Error: unclosed ring for input: 'COc1ccc(CC(NC(=O)C2CCCCC2)C(=O)NC(CC)C(=O)NCc2ccc(C(N)N)cc2)'


## Summary

In [5]:

original_property_path = f"property/{file_name}.csv"
original_update_mapping_path = f"refinement/{file_name}.csv"
updated_property_path = f"property/{file_name}_update.csv"

# Load data
original_property_df = pd.read_csv(original_property_path)
updated_property_df = pd.read_csv(updated_property_path)
update_mapping_df = pd.read_csv(original_update_mapping_path)

# Collect results here
results = []
for index, row in update_mapping_df.iterrows():
    original_smiles = row['SMILES'].strip()
    updated_smiles = row['Updated_SMILES'].strip()
    property_name = row['Property'].strip().replace("'", "")

    # Normalize property name if needed
    if not any(suffix in property_name for suffix in ["Predictions", "Interpretation", "Probability"]):
        property_name += " Predictions"

    # Validate SMILES presence
    if original_smiles not in original_property_df['SMILES'].values:
        print(f"[Missing] Original SMILES not found: {original_smiles}")
        continue
    if updated_smiles not in updated_property_df['SMILES'].values:
        print(f"[Missing] Updated SMILES not found: {updated_smiles}")
        continue

    try:
        original_value = original_property_df.loc[
            original_property_df['SMILES'] == original_smiles, property_name
        ].values[0]

        updated_value = updated_property_df.loc[
            updated_property_df['SMILES'] == updated_smiles, property_name
        ].values[0]

        # Determine direction
        if isinstance(original_value, (int, float)) and isinstance(updated_value, (int, float)):
            if original_value < updated_value:
                direction = "up"
            elif original_value > updated_value:
                direction = "down"
            else:
                direction = "-"

        elif isinstance(original_value, str) and isinstance(updated_value, str):
            if original_value == updated_value:
                direction = "-"
            elif original_value == 'Toxic' and updated_value == 'Safe':
                direction = "safe"
            elif original_value == 'Safe' and updated_value == 'Toxic':
                direction = "toxic"
            elif original_value == 'Inhibitor' and updated_value != 'Inhibitor':
                direction = "Non-Inhibitor"
        # else:
        #     direction = "non-numerical"

        # Append to result list
        results.append({
            "Property": property_name,
            "Original_SMILES": original_smiles,
            "Updated_SMILES": updated_smiles,
            "Original_Value": original_value,
            "Updated_Value": updated_value,
            "Direction": direction
        })

    except KeyError as e:
        print(f"[KeyError] Property not found: {property_name}")
        continue
    except IndexError:
        print(f"[IndexError] Value missing for SMILES or property: {original_smiles}, {updated_smiles}")
        continue

# Convert results to DataFrame
result_df = pd.DataFrame(results)

# (Optional) Save to CSV
result_df.to_csv(f"{save_directory}/compare_{file_name}.csv", index=False)

# Preview the result
result_df

[KeyError] Property not found: Toxicity/AMES Mutagenesis Predictions
[Missing] Updated SMILES not found: O=C(O)c1ccc(N2CCN(c3ccc(Oc4cnco4)c3)CC2)cc1
[Missing] Updated SMILES not found: CN(C)S(=O)(=O)NC(=O)c1ccc(N2CCN(C[C@H]3CC[C@@H](C(=O)O)CC3)[C@@H]2Oc2cnc(N)cc2Cl)
[Missing] Updated SMILES not found: COc1ccc(OCCNC(=O)c2cc3cc(O)ccc3n2Cc2ccccc2)
[KeyError] Property not found: Invalid SMILES Predictions
[KeyError] Property not found: Toxicity/AMES Mutagenesis Predictions
[KeyError] Property not found: Toxicity Predictions
[Missing] Updated SMILES not found: COc1ccc(CC(NC(=O)C2CCCCC2)C(=O)NC(CC)C(=O)NCc2ccc(C(N)N)cc2)


,Property,Original_SMILES,Updated_SMILES,Original_Value,Updated_Value,Direction
0,[Absorption/Caco-2 (logPaap)] Predictions,O=C(NS(=O)(=O)c1ccc(N2CCN(Cc3ccccc3Cl)CC2=O)cc...,O=C(NS(=O)(=O)c1ccc(N2CCN(Cc3ccccc3Cl)CC2=O)cc...,-5.76,-5.79,down
1,[Absorption/Caco-2 (logPaap)] Predictions,CC(C)CNc1ccc(S(=O)(=O)NC(=O)c2ccc(N3CCN(C)C(=O...,CC(C)CNc1ccc(S(=O)(=O)NC(=O)c2ccc(N3CCN(C)C(=O...,-5.59,-5.74,down
2,[Absorption/Caco-2 (logPaap)] Predictions,O=C(NS(=O)(=O)c1ccc(N2CCN(CC3CCC3)CC2=O)cc1)c1...,O=C(Nc1ccc(N2CCN(CC3CCC3)CC2=O)cc1)c1ccc(Cl)nc1Cl,-4.95,-4.54,up
3,[Toxicity/hERG Blockers] Predictions,CN(C)CCNc1ccc(S(=O)(=O)NC(=O)c2ccc(N3CCN(C/C=C...,CN(C)CCNc1ccc(S(=O)(=O)NC(=O)c2ccc(N3CCN(C/C=C...,Toxic,Toxic,-
4,[Absorption/Caco-2 (logPaap)] Predictions,COc1ccccc1S(=O)(=O)NC(=O)c1ccc(N2CCN(C)C(=O)C2...,COc1ccccc1S(=O)(=O)NC(=O)c1ccc(N2CCN(C)C(=O)C2...,-5.21,-5.21,-
...,...,...,...,...,...,...
86,[Absorption/Caco-2 (logPaap)] Predictions,COc1ccc(CN2CCc3c(cc(C(=O)N4CCOC5(CCNC5)C4)n3C)...,CCc1ccc(CN2CCc3c(cc(C(=O)N4CCOC5(CCNC5)C4)n3C)...,-5.12,-5.14,down
87,[Absorption/Caco-2 (logPaap)] Predictions,CC1CNC(CC(=O)Nc2ccc(-c3ccccc3)cc2)c2c(nc3ccccn...,CC1CNC(CC(=O)Nc2ccc(-c3ccccc3)cc2)c2c(nc3ccccn...,-5.1,-5.1,-
88,[Toxicity/AMES Mutagenesis] Predictions,NC(=O)c1[nH]c(=Nc2cccc3ccccc23)sc1C(=O)c1ccccc1F,NC(=O)c1[nH]c(=Nc2cccc3ccccc23)sc1C(=O)c1ccccc1F,Toxic,Toxic,-
89,[General Properties/Log(P)] Predictions,N#Cc1ccc(-c2ccc(CN(C(=O)Cc3ccccc3)c3ccc4c(c3)C...,N#Cc1ccc(-c2ccc(CN(C(=O)Cc3cc(O)ccc3)c3ccc4c(c...,6.63,6.16,down


## Absorption property comparison

In [6]:
# === [Caco-2 Permeability Statistics] ===
specific_property = "[Absorption/Caco-2 (logPaap)] Predictions"
threshold_value = -5.0

# Filter by property
property_df = result_df[result_df['Property'] == specific_property]
print(f"\nTotal entries for '{specific_property}': {len(property_df)}")

# Count direction values
up_count = property_df[property_df['Direction'] == 'up'].shape[0]
down_count = property_df[property_df['Direction'] == 'down'].shape[0]
no_change_count = property_df[property_df['Direction'] == '-'].shape[0]
print(f"Up: {up_count}, Down: {down_count}, No Change: {no_change_count}")

# Filter by threshold on Original_Value
numeric_df = property_df.copy()
numeric_df['Original_Value'] = pd.to_numeric(numeric_df['Original_Value'], errors='coerce')
filtered_df = numeric_df[numeric_df['Original_Value'] < threshold_value]

print(f"\nFiltered results for '{specific_property}' with Original_Value < {threshold_value}: {len(filtered_df)} entries")

# Count direction values in filtered set
filtered_up_count = filtered_df[filtered_df['Direction'] == 'up'].shape[0]
filtered_down_count = filtered_df[filtered_df['Direction'] == 'down'].shape[0]
filtered_no_change_count = filtered_df[filtered_df['Direction'] == '-'].shape[0]
print(f"Filtered Up: {filtered_up_count}, Filtered Down: {filtered_down_count}, Filtered No Change: {filtered_no_change_count}")
result = {
    "Property": specific_property,
    "Total": len(property_df),
    "Up": up_count,
    "Down": down_count,
    "No Change": no_change_count,
    "Filter Threshold": threshold_value,
    "Filtered Count": len(filtered_df),
    "Filtered Up": filtered_up_count,
    "Filtered Down": filtered_down_count,
    "Filtered No Change": filtered_no_change_count
}
summary_df = pd.DataFrame([result])
result.pop('Property')
report[specific_property] = result


Total entries for '[Absorption/Caco-2 (logPaap)] Predictions': 57
Up: 25, Down: 15, No Change: 17

Filtered results for '[Absorption/Caco-2 (logPaap)] Predictions' with Original_Value < -5.0: 39 entries
Filtered Up: 17, Filtered Down: 9, Filtered No Change: 13


## Toxicity property comparison

In [7]:
# === [Toxicity Statistics] ===
specific_property = "Toxicity"
threshold_value = "Toxic"

# Filter by property (partial match)
property_df = result_df[result_df['Property'].str.contains(specific_property, na=False)]
print(f"\nTotal entries for '{specific_property}': {len(property_df)}")

# Count direction values
safe_count = property_df[property_df['Direction'] == 'safe'].shape[0]
toxic_count = property_df[property_df['Direction'] == 'toxic'].shape[0]
no_change_count = property_df[property_df['Direction'] == '-'].shape[0]
print(f"Safe: {safe_count}, Toxic: {toxic_count}, No Change: {no_change_count}")

# Filter by exact match on Original_Value
filtered_df = property_df[property_df['Original_Value'] == threshold_value]
print(f"\nFiltered results for '{specific_property}' with Original_Value = '{threshold_value}': {len(filtered_df)} entries")

# Count direction values in filtered set
filtered_safe_count = filtered_df[filtered_df['Direction'] == 'safe'].shape[0]
filtered_toxic_count = filtered_df[filtered_df['Direction'] == 'toxic'].shape[0]
filtered_no_change_count = filtered_df[filtered_df['Direction'] == '-'].shape[0]
print(f"Filtered Safe: {filtered_safe_count}, Filtered Toxic: {filtered_toxic_count}, Filtered No Change: {filtered_no_change_count}")

result = {
    "Property": specific_property,
    "Total": len(property_df),
    "Up": safe_count,
    "Down": toxic_count,
    "No Change": no_change_count,
    "Filter Threshold": threshold_value,
    "Filtered Count": len(filtered_df),
    "Filtered Up": filtered_safe_count,
    "Filtered Down": filtered_toxic_count,
    "Filtered No Change": filtered_no_change_count
}
summary_df = pd.DataFrame([result])
result.pop('Property')
report[specific_property] = result


Total entries for 'Toxicity': 25
Safe: 6, Toxic: 0, No Change: 19

Filtered results for 'Toxicity' with Original_Value = 'Toxic': 25 entries
Filtered Safe: 6, Filtered Toxic: 0, Filtered No Change: 19


# Rule-based Evaluation


In [8]:
def process_dataset_with_agent(file_path: str) -> pd.DataFrame:
    """
    Reads the dataset and applies the LLM-driven analysis to each entry.

    Args:
        file_path (str): Path to the dataset file.
        protein (str): Target protein for drug analysis.
        objective (str): Target objective for drug analysis.
        agent: The LLM agent object.

    Returns:
        pd.DataFrame: DataFrame with the original data and LLM assessments.
    """
    # Read dataset
    df = pd.read_csv(file_path)

    # Prepare list to store output
    results = []

    # Iterate through each row
    for _, row in df.iterrows():
        # rule-based evaluation
        entry = row.to_dict()

        qed_score = QED.qed(Chem.MolFromSmiles(entry['SMILES']))
        report = rule_based_evaluation(entry)
        report['QED'] = qed_score
        # binding affinity determination
        affinity = entry["Affinity [pKd]"]
        report['Affinity'] = affinity
        
        results.append({"Method": entry["Method"], **report})

    # Create and return the new DataFrame
    return pd.DataFrame(results)


In [9]:
drug_likeness_rules = ["lipinski_rule_of_5",	
                      "veber_rule",	
                      "ghose_filter",	
                      "rule_of_3",
                      "oprea_lead_like"]

for suffix in ["", "_update"]:
    # Define property file path
    property_file = f"property/{file_name}{suffix}.csv"
    
    # Process the dataset
    processed_data = process_dataset_with_agent(property_file)
    #processed_data['Method'] = method + suffix
    # Save the processed data
    save_path = os.path.join(save_directory, os.path.basename(property_file))
    num_passed_rules = processed_data[drug_likeness_rules].sum(axis=1)
    processed_data['num_passed_rules'] = num_passed_rules
    print(save_path) 
    processed_data.to_csv(save_path, index=False)
    
    # Compute and store statistics
    stats_key = f"rule_eval{suffix}" if suffix else "rule_eval"
    report[stats_key] = processed_data.describe().to_dict()

analysis_result/combined_sampling.csv
analysis_result/combined_sampling_update.csv


In [10]:
with open(f"{save_directory}/report_{file_name}.json", "w") as f:
    json.dump(report, f, indent=4)

# Collective Analysis

In [11]:
original_data = pd.read_csv("analysis_result/combined_sampling.csv")
first_update_data = pd.read_csv("analysis_result/combined_sampling_update.csv")
second_update_data = pd.read_csv("analysis_result/combined_sampling_update_update.csv")
first_update_result = json.load(open('analysis_result/report_combined_sampling.json'))
second_update_result = json.load(open('analysis_result/report_combined_sampling_update.json'))

## Toxicity

In [ ]:
# --- Plot: First vs Second Refinement ---

categories = ['Improved', 'Declined', 'Unchanged']
x = np.arange(len(categories))
width = 0.35  # width of the bars

# First refinement ratios
tox_data1 = first_update_result['Toxicity']
count1 = tox_data1['Filtered Count'] 
ratios1 = [
    tox_data1['Filtered Up'] / count1,
    tox_data1['Filtered Down'] / count1,
    tox_data1['Filtered No Change'] / count1
]
print(ratios1)
# Second refinement ratios
tox_data2 = second_update_result['Toxicity']
count2 = tox_data2['Filtered Count']
ratios2 = [
    tox_data2['Filtered Up'] / count2,
    tox_data2['Filtered Down'] / count2,
    tox_data2['Filtered No Change'] / count2
]
print(ratios2)
# Colors (updated to be more distinguishable)
color1 = '#66c2a5'  # soft green
color2 = '#fc8d62'  # soft orange

plt.figure(figsize=(5, 4))
bars1 = plt.bar(x - width/2, ratios1, width, label=f'1st Refinement ({count1})', color=color1, alpha=0.8)
bars2 = plt.bar(x + width/2, ratios2, width, label=f'2nd Refinement ({count2})', color=color2, alpha=0.8)

# Set labels and ticks
plt.ylabel('Ratio', fontsize=15)
plt.xticks(x, categories, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc='upper left', framealpha=0.4)
plt.ylim(0, 1.0)
plt.tight_layout()
plt.savefig('figures/toxicity_ratios_1st_vs_2nd.png', dpi=300, bbox_inches='tight')
plt.show()

[0.24, 0.0, 0.76]
[0.2, 0.0, 0.8]


## Permeability

In [13]:
categories = ['Improved', 'Declined', 'Unchanged']
x = np.arange(len(categories))
width = 0.35  # width of each bar group

# --- Extract from first refinement ---
per_data_1st = first_update_result['[Absorption/Caco-2 (logPaap)] Predictions']
total_count_1st = per_data_1st['Total']
filtered_count_1st = per_data_1st['Filtered Count']

total_ratios_1st = [
    per_data_1st['Up'] / total_count_1st,
    per_data_1st['Down'] / total_count_1st,
    per_data_1st['No Change'] / total_count_1st
]
print(total_ratios_1st)
filtered_ratios_1st = [
    per_data_1st['Filtered Up'] / filtered_count_1st,
    per_data_1st['Filtered Down'] / filtered_count_1st,
    per_data_1st['Filtered No Change'] / filtered_count_1st
]

# --- Extract from second refinement ---
per_data_2nd = second_update_result['[Absorption/Caco-2 (logPaap)] Predictions']
total_count_2nd = per_data_2nd['Total']
filtered_count_2nd = per_data_2nd['Filtered Count']

total_ratios_2nd = [
    per_data_2nd['Up'] / total_count_2nd,
    per_data_2nd['Down'] / total_count_2nd,
    per_data_2nd['No Change'] / total_count_2nd
]
print(total_ratios_2nd)
filtered_ratios_2nd = [
    per_data_2nd['Filtered Up'] / filtered_count_2nd,
    per_data_2nd['Filtered Down'] / filtered_count_2nd,
    per_data_2nd['Filtered No Change'] / filtered_count_2nd
]

[0.43859649122807015, 0.2631578947368421, 0.2982456140350877]
[0.5230769230769231, 0.16923076923076924, 0.3076923076923077]


In [14]:
# --- Plot: Total Data ---
plt.figure(figsize=(5, 4))
plt.bar(x - width/2, total_ratios_1st, width, label=f'1st Refinement ({total_count_1st})', color='#66c2a5', alpha=0.8)
plt.bar(x + width/2, total_ratios_2nd, width, label=f'2nd Refinement ({total_count_2nd})', color='#fc8d62', alpha=0.8)

plt.ylabel('Ratio', fontsize=15)
plt.xticks(x, categories, fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc='upper right', framealpha=0.4)
plt.ylim(0, 1.0)
plt.tight_layout()
plt.savefig('figures/permeability_ratios_total_1st_vs_2nd.png', dpi=300, bbox_inches='tight')
# plt.show()

## QED

In [15]:
qed_original = original_data['QED']
qed_first_update = first_update_data['QED']
qed_second_update = second_update_data['QED']
bw_adjust_value = 0.4

# Create a plot with KDE-style distribution curves using seaborn
plt.figure(figsize=(9, 6))

sns.kdeplot(qed_original, label='Original', linewidth=2, bw_adjust=bw_adjust_value)
sns.kdeplot(qed_first_update, label='1st Refinement', linewidth=2, bw_adjust=bw_adjust_value)
sns.kdeplot(qed_second_update, label='2nd Refinement', linewidth=2, bw_adjust=bw_adjust_value)

plt.xlabel('QED Value', fontsize=20)
plt.ylabel('Density', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=18)
plt.tight_layout()
plt.savefig('figures/qed_distribution.png', dpi=300, bbox_inches='tight')

In [18]:
# number of entries QED > 0.6 
qed_threshold = 0.6
qed_original_count = (qed_original > qed_threshold).sum()
qed_first_update_count = (qed_first_update > qed_threshold).sum()
qed_second_update_count = (qed_second_update > qed_threshold).sum()

print(f"QED > {qed_threshold}: Original: {qed_original_count}, 1st Update: {qed_first_update_count}, 2nd Update: {qed_second_update_count}")

QED > 0.6: Original: 23, 1st Update: 33, 2nd Update: 43


In [16]:
rule_original = original_data['num_passed_rules']
rule_first_update = first_update_data['num_passed_rules']
rule_second_update = second_update_data['num_passed_rules']

# Combine data into a long-form DataFrame for seaborn
df_bar = pd.DataFrame({
    'Original': rule_original,
    '1st Refinement': rule_first_update,
    '2nd Refinement': rule_second_update
})

df_long = df_bar.melt(var_name='Dataset', value_name='Number of Rules Passed')

# Plot
plt.figure(figsize=(8, 6))
sns.countplot(data=df_long, x='Number of Rules Passed', hue='Dataset')

plt.xlabel('Number of Rules Passed', fontsize=20)
plt.ylabel('Count', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=18)

plt.tight_layout()
plt.savefig('figures/rules_passed_distribution.png', dpi=300, bbox_inches='tight')

In [17]:
#. number of moleulce passing >=4 rules
num_passed_4_rules = df_long[df_long['Number of Rules Passed'] >= 4].groupby('Dataset').size()
print("\nNumber of molecules passing >= 4 rules:")
print(num_passed_4_rules)


Number of molecules passing >= 4 rules:
Dataset
1st Refinement    49
2nd Refinement    55
Original          34
dtype: int64
